**В этом блокноте я тренировался с агрегацией и группировкой данных**

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 

In [2]:
df = sns.load_dataset('titanic')

Каждая строка набора данных представляет одного человека. Столбцы описывают различные атрибуты, включая то, выжили ли люди (`survived`), их возраст (`age`), класс пассажира (`pclass`), пол (`sex`) и стоимость проезда (`fare`).

In [3]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Агрегирование

In [4]:
df['fare'].agg(['sum','mean']) # сумма и среднее по столбцу стоимости билета 

sum     28693.949300
mean       32.204208
Name: fare, dtype: float64

In [5]:
df.agg({'fare' : ['sum','mean'],
        'sex' : ['count']})

,fare,sex
count,NaN,891.0
mean,32.204208,NaN
sum,28693.949300,NaN


In [6]:
df.agg(fare_sum = ('fare','sum'),
       fare_mean = ('fare','mean'),
       sex_count = ('sex','count'))

,fare,sex
fare_sum,28693.949300,NaN
fare_mean,32.204208,NaN
sex_count,NaN,891.0


## Groupby

Наиболее распространенными встроенными функциями агрегирования являются базовые математические функции, включая сумму (`sum`), среднее значение (`mean`), медианное значение (`median`), минимум (`minimum`), максимум (`maximum`), стандартное отклонение (`standard deviation`), дисперсию (`variance`), среднее абсолютное отклонение (`mean absolute deviation`) и произведение (`product`).

In [7]:
agg_func_math = {
    'fare': ['sum', 'mean', 'median', 'min', 'max', 'std', 'var', 'mad', 'prod']
}

In [8]:
df.groupby(['embark_town']).agg(agg_func_math).round(2)

fare                                                     \
                  sum   mean median   min     max    std      var    mad   
embark_town                                                                
Cherbourg    10072.30  59.95  29.70  4.01  512.33  83.91  7041.39  53.02   
Queenstown    1022.25  13.28   7.75  6.75   90.00  14.19   201.30   7.87   
Southampton  17439.40  27.08  13.00  0.00  263.00  35.89  1287.95  21.30   

                            
                      prod  
embark_town                 
Cherbourg    6.193716e+250  
Queenstown    6.458671e+78  
Southampton   0.000000e+00

полезный трюк - использовать `describe` для одновременного выполнения нескольких встроенных агрегаторов:



In [9]:
agg_func_describe = {'fare' : ['describe']}

In [10]:
df.groupby(['embark_town']).agg(agg_func_describe).round(2)

fare                                                
            describe                                                
               count   mean    std   min    25%    50%   75%     max
embark_town                                                         
Cherbourg      168.0  59.95  83.91  4.01  13.70  29.70  78.5  512.33
Queenstown      77.0  13.28  14.19  6.75   7.75   7.75  15.5   90.00
Southampton    644.0  27.08  35.89  0.00   8.05  13.00  27.9  263.00

## Подсчёт

После базовой математики подсчет (counting) является следующим наиболее распространенным агрегированием для сгруппированных данных.

Вот три примера подсчета:

In [11]:
agg_func_count = {'embark_town' : ['count','nunique','size']}

In [12]:
df.groupby(['deck']).agg(agg_func_count)

embark_town             
           count nunique size
deck                         
A             15       2   15
B             45       2   47
C             59       3   59
D             33       2   33
E             32       3   32
F             13       3   13
G              4       1    4

Главное отличие, о котором следует помнить, заключается в том, что `count` не включает значения `NaN`, тогда как `size` их включает. В зависимости от набора данных это различие может оказаться полезным

Кроме того, функция `nunique` исключит значения `NaN` из уникальных счетчиков.

Сейчас мы можем выбрать самую высокую и самую низкую стоимость билета в зависимости от города, в котором совершили посадку пассажиры Титаника.

Следует помнить один важный момент: нужно сначала отсортировать данные, чтобы в качестве `first`  и `last` были выбраны максимальное и минимальное значения.

In [13]:
agg_func_selection = {'fare' : ['first','last']}

In [14]:
df.sort_values(by=['fare'],
               ascending=False).groupby(['embark_town']).agg(agg_func_selection)

fare        
                first    last
embark_town                  
Cherbourg    512.3292  4.0125
Queenstown    90.0000  6.7500
Southampton  263.0000  0.0000

Другой вариант использовать `idxmax` и `idxmin` для выбора значения индекса, соответствующегомаксимальному и минимальному значениям.

In [15]:
agg_func_min_max = {'fare' : ['idxmax','idxmin']}

In [16]:
df.groupby(['embark_town']).agg(agg_func_min_max)

fare       
            idxmax idxmin
embark_town              
Cherbourg      258    378
Queenstown     245    143
Southampton     27    179

Проверим :

In [17]:
df.loc[[258,378]]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
258,1,1,female,35.0,0,0,512.3292,C,First,woman,False,NaN,Cherbourg,yes,True
378,0,3,male,20.0,0,0,4.0125,C,Third,man,True,NaN,Cherbourg,no,True


In [18]:
# просмотр строк с максимальной стоимостью проезда(fare)
df.loc[df.groupby('class')['fare'].idxmax()]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
258,1,1,female,35.0,0,0,512.3292,C,First,woman,False,NaN,Cherbourg,yes,True
72,0,2,male,21.0,0,0,73.5000,S,Second,man,True,NaN,Southampton,no,True
159,0,3,male,NaN,8,2,69.5500,S,Third,man,True,NaN,Southampton,no,False


расчет `mode` и `skew` данных для стоимости проезда.

In [19]:
from scipy.stats import skew, mode

In [20]:
agg_func_stats = {'fare' : [skew,mode,pd.Series.mode]}

In [22]:
df.groupby(['embark_town']).agg(agg_func_stats)

fare                          
                 skew              mode    mode
embark_town                                    
Cherbourg    3.305112  ([7.2292], [15])  7.2292
Queenstown   4.265111    ([7.75], [30])  7.7500
Southampton  3.640276    ([8.05], [43])  8.0500

Интересны результаты вычисления моды (`mode`). Функция `mode` из `scipy.stats` возвращает наиболее часто встречающееся значение, а также количество вхождений.

Если нужно наиболее частое значение, то используем `pd.Series.mode`.

## Работа с текстом

In [25]:
agg_func_text = {'deck' : ['nunique',
                           mode,
                           set]}

In [26]:
df.groupby(['class']).agg(agg_func_text)

deck                                   
       nunique         mode                   set
class                                            
First        5  ([C], [59])  {nan, B, C, A, D, E}
Second       3   ([F], [8])        {nan, D, E, F}
Third        3   ([F], [5])        {nan, G, E, F}

## Пользовательские функции

In [27]:
from functools import partial 

In [37]:
# например посчитаем 25 процентиль и сравним с готовым решением 
q_25 = partial(pd.Series.quantile, q=0.25) # возвращает обёртку над pd.Series.quantile()

In [29]:
# зададим наименование будущего столбца 
q_25.__name__ = '25%'

In [38]:
# определяем нашу функцию, которая пойдёт оберткой для quantile
def percentile_25(x):
    return x.quantile(.25)

In [40]:
lambda_25 = lambda x : x.quantile(.25)

In [41]:
lambda_25.__name__ = 'lambda_25%'

In [42]:
agg_func = {
    'fare' : [q_25,
              percentile_25,
              lambda_25,
              lambda x : x.quantile(.25)]
}

In [43]:
df.groupby(['embark_town']).agg(agg_func).round(2)

fare                                    
            quantile percentile_25 lambda_25% <lambda_0>
embark_town                                             
Cherbourg      13.70         13.70      13.70      13.70
Queenstown      7.75          7.75       7.75       7.75
Southampton     8.05          8.05       8.05       8.05

## Примеры пользовательских функций 

Как показано выше, существует несколько подходов к разработке пользовательских функций агрегирования.

В большинстве случаев функции представляют собой легкие обертки (`wrappers`) для встроенных функций pandas. Они нужны, т.к. нет возможности передать аргументы в агрегаты (`aggregations`).

In [44]:
# для подсчёта нулевых значений 
def count_nulls(s):
    return s.size - s.count()

Если нужноё включить значения `NaN` в свои уникальные счетчики, необходимо указать параметр `dropna=False` у функции `nunique`.

In [45]:
def unique_nan(s):
    return s.nunique(dropna=False)

In [46]:
# результат всех функций 
agg_func_custom_count = {
    'embark_town' : ['count', 
                     'nunique', 
                     'size', 
                     unique_nan, 
                     count_nulls, set]}


In [47]:
df.groupby(['deck']).agg(agg_func_custom_count)

embark_town                                      \
           count nunique size unique_nan count_nulls   
deck                                                   
A             15       2   15          2           0   
B             45       2   47          3           2   
C             59       3   59          3           0   
D             33       2   33          2           0   
E             32       3   32          3           0   
F             13       3   13          3           0   
G              4       1    4          1           0   

                                            
                                       set  
deck                                        
A                 {Cherbourg, Southampton}  
B            {nan, Cherbourg, Southampton}  
C     {Cherbourg, Queenstown, Southampton}  
D                 {Cherbourg, Southampton}  
E     {Cherbourg, Queenstown, Southampton}  
F     {Cherbourg, Queenstown, Southampton}  
G                            {Southampton}

In [48]:
from sparklines import sparklines

In [49]:
def sparkline_str(x):
    bins = np.histogram(x)[0]
    sl = ''.join(sparklines(bins))
    return sl

In [51]:
df.groupby(['class', 'embark_town']).agg({'fare' : [sparkline_str]})

fare
                   sparkline_str
class  embark_town              
First  Cherbourg      █▇▂▁▃▁▁▁▁▂
       Queenstown     ▁▁▁▁▁█▁▁▁▁
       Southampton    ▃█▄▃▂▂▁▁▂▂
Second Cherbourg      █▄▁▁▄▂▄▁▄▅
       Queenstown     ▁▁▁▁▁█▁▁▁▁
       Southampton    ▂█▂▅▁▂▁▁▁▁
Third  Cherbourg      ▁█▃▂▁▄▃▁▂▂
       Queenstown     █▁▁▂▁▁▁▂▁▂
       Southampton    ▁█▂▂▂▁▁▁▁▁

In [52]:
# суммарная стоимость для 10 первых и последних пассажиров 

def top10_sum(x):
    return x.nlargest(10).sum()

def bottom10_sum(x):
    return x.nsmallest(10).sum()

In [53]:
agg_func_top_bottom_sum = {
    'fare' : [top10_sum,
              bottom10_sum]
}

In [54]:
df.groupby('class').agg(agg_func_top_bottom_sum)

fare             
        top10_sum bottom10_sum
class                         
First   3361.2584     108.3709
Second   622.2376      42.0000
Third    656.3374      36.1291

## Пользовательские функции с несколькими столбцами

In [55]:
def summary(x):
    result = {
        'fare_sum' : x['fare'].sum(),
        'fare_mean' : x['fare'].mean(),
        'fare_range' : x['fare'].max() - x['fare'].min()
    }
    
    return pd.Series(result).round(0)

In [58]:
df.groupby(['class']).apply(summary)

,fare_sum,fare_mean,fare_range
class,,,
First,18177.0,84.0,512.0
Second,3802.0,21.0,74.0
Third,6715.0,14.0,70.0


Использование `apply` с `groupby` дает максимальную гибкость. Однако есть и обратная сторона. Функция `apply` работает медленно, поэтому этот подход следует использовать с осторожностью.

## Работа с групповыми объектами 

После группировки и агрегирования данных можно выполнять дополнительные вычисления для сгруппированных объектов.

В следующем примере определим, какой процент от общего количества проданных билетов можно отнести к каждой комбинации `embark_town` и `class`.

In [61]:
df.groupby(['embark_town','class']).agg({'fare' : 'sum'}).assign(pct_total = lambda x : x / x.sum())

fare  pct_total
embark_town class                       
Cherbourg   First   8901.0750   0.311947
            Second   431.0917   0.015108
            Third    740.1295   0.025939
Queenstown  First    180.0000   0.006308
            Second    37.0500   0.001298
            Third    805.2043   0.028219
Southampton First   8936.3375   0.313183
            Second  3333.7000   0.116833
            Third   5169.3613   0.181165

можно сделать проще с использованием кросс-таблицы `pd.crosstab`

In [62]:
pd.crosstab(df['embark_town'],
            df['class'],
            values=df['fare'],
            aggfunc='sum',
            normalize=True)

class,First,Second,Third
embark_town,,,
Cherbourg,0.311947,0.015108,0.025939
Queenstown,0.006308,0.001298,0.028219
Southampton,0.313183,0.116833,0.181165


функции агрегации также можно комбинировать со сводными таблицами (`pivot tables`).

In [63]:
pd.pivot_table(data=df,
               index=['embark_town'],
               columns=['class'],
               aggfunc=agg_func_top_bottom_sum)

fare                                                  
            bottom10_sum                     top10_sum                    
class              First    Second    Third      First    Second     Third
embark_town                                                               
Cherbourg       282.9957  172.2041  68.2500  3239.3542  334.6084  196.7457
Queenstown      180.0000   37.0500  73.5916   180.0000   37.0500  264.5750
Southampton     108.3709   42.0000  39.6291  2237.5251  614.5000  656.3374

Иногда необходимо выполнить множество группировок (multiple groupby), чтобы ответить на вопрос. Например, если мы хотим увидеть кумулятивную сумму стоимости билетов, мы можем сгруппировать и агрегировать по городу (town) и классу (class), затем сгруппировать полученный объект и вычислить кумулятивную сумму (cumulative sum):

In [64]:
fare_group = df.groupby(['embark_town', 'class']).agg({'fare' : 'sum'})
fare_group

fare
embark_town class            
Cherbourg   First   8901.0750
            Second   431.0917
            Third    740.1295
Queenstown  First    180.0000
            Second    37.0500
            Third    805.2043
Southampton First   8936.3375
            Second  3333.7000
            Third   5169.3613

In [66]:
fare_group.groupby(level=0).cumsum()

fare
embark_town class             
Cherbourg   First    8901.0750
            Second   9332.1667
            Third   10072.2962
Queenstown  First     180.0000
            Second    217.0500
            Third    1022.2543
Southampton First    8936.3375
            Second  12270.0375
            Third   17439.3988

![title](multiple-groupby.png)

## Пример с данными о продажах

В следующем примере резюмируем ежедневные данные о продажах и преобразуем их в совокупное ежедневное и ежеквартальное представление.

In [67]:
sales = pd.read_excel('https://github.com/chris1610/pbpython/blob/master/data/2018_Sales_Total_v2.xlsx?raw=True')
sales.head()

,account number,name,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,B1-20000,39,86.69,3380.91,2018-01-01 07:21:51
1,714466,Trantow-Barrows,S2-77896,-1,63.16,-63.16,2018-01-01 10:00:47
2,218895,Kulas Inc,B1-69924,23,90.70,2086.10,2018-01-01 13:24:58
3,307599,"Kassulke, Ondricka and Metz",S1-65481,41,21.05,863.05,2018-01-01 15:05:22
4,412290,Jerde-Hilpert,S2-34077,6,83.21,499.26,2018-01-01 23:26:55


In [68]:
daily_sales = sales.groupby([pd.Grouper(key='date',
                                        freq='D')]).agg(daily_sales=('ext price','sum')).reset_index()

In [69]:
daily_sales.head()

,date,daily_sales
0,2018-01-01,6766.16
1,2018-01-02,1551.91
2,2018-01-03,4278.96
3,2018-01-04,6044.10
4,2018-01-05,1971.94


In [70]:
daily_sales['quarter_sales'] = daily_sales.groupby(pd.Grouper(key='date',
                                                              freq='Q')).agg({'daily_sales' : 'cumsum'})
daily_sales.head()

,date,daily_sales,quarter_sales
0,2018-01-01,6766.16,6766.16
1,2018-01-02,1551.91,8318.07
2,2018-01-03,4278.96,12597.03
3,2018-01-04,6044.10,18641.13
4,2018-01-05,1971.94,20613.07


Чтобы получить хорошее представление о том, что происходит, нужно взглянуть на границу квартала (с конца марта по начало апреля):

![title](cumulative_total.png)

Если нужно просто получить совокупный (cumulative) квартальный итог, вы можете связать несколько функций groupby.

In [71]:
sales.groupby(
        pd.Grouper(key='date',
                   freq='D')).agg(
                                  daily_sales=('ext price',
                                               'sum')).groupby(
                                                               pd.Grouper(freq='Q')).agg(
                                                                                         {'daily_sales' : 'cumsum'}).rename(
                                                                                                                            columns={'daily_sales' : 'quarterly_sales'})

,quarterly_sales
date,
2018-01-01,6766.16
2018-01-02,8318.07
2018-01-03,12597.03
2018-01-04,18641.13
2018-01-05,20613.07
...,...
2018-12-27,480817.47
2018-12-28,484389.92
2018-12-29,489227.01


В этом примере я включил именованный подход агрегации (named aggregation approach), чтобы переименовать переменную и уточнить, что теперь это ежедневные продажи. Затем я снова группирую и использую совокупную (cumulative) сумму, чтобы получить текущую сумму за квартал. Наконец, я переименовал столбец в квартальные продажи (quarterly sales).

## Сглаживание иерархических индексов столбцов

In [72]:
df.groupby(['embark_town', 'class']).agg({'fare' : ['sum', 'mean']}).round()

fare       
                       sum   mean
embark_town class                
Cherbourg   First   8901.0  105.0
            Second   431.0   25.0
            Third    740.0   11.0
Queenstown  First    180.0   90.0
            Second    37.0   12.0
            Third    805.0   11.0
Southampton First   8936.0   70.0
            Second  3334.0   20.0
            Third   5169.0   15.0

In [73]:
multi_df = df.groupby(['embark_town', 'class'], as_index=False).agg({'fare' : ['sum', 'mean']})
multi_df.head()

embark_town   class       fare            
                             sum        mean
0   Cherbourg   First  8901.0750  104.718529
1   Cherbourg  Second   431.0917   25.358335
2   Cherbourg   Third   740.1295   11.214083
3  Queenstown   First   180.0000   90.000000
4  Queenstown  Second    37.0500   12.350000

In [74]:
multi_df.columns = ['_'.join(col).rstrip('_') for col in multi_df.columns.values]
multi_df.round(2)

,embark_town,class,fare_sum,fare_mean
0,Cherbourg,First,8901.07,104.72
1,Cherbourg,Second,431.09,25.36
2,Cherbourg,Third,740.13,11.21
3,Queenstown,First,180.00,90.00
4,Queenstown,Second,37.05,12.35
5,Queenstown,Third,805.20,11.18
6,Southampton,First,8936.34,70.36
7,Southampton,Second,3333.70,20.33
8,Southampton,Third,5169.36,14.64


Вот изображение, показывающее, как выглядит сглаженный DataFrame:

![title](column_flatten.png)

## Промежуточные итоги

Вот как вы можете суммировать `fares` по `class`, `embark_town` и `sex` с промежуточным итогом на каждом уровне, а также общим итогом внизу:

In [75]:
import sidetable

In [76]:
df.groupby(['class',
            'embark_town',
            'sex']).agg({'fare' : 'sum'}).stb.subtotal()

fare
class       embark_town       sex                                        
First       Cherbourg         female                            4972.5333
                              male                              3928.5417
                              First | Cherbourg - subtotal      8901.0750
            Queenstown        female                              90.0000
                              male                                90.0000
                              First | Queenstown - subtotal      180.0000
            Southampton       female                            4753.2917
                              male                              4183.0458
                              First | Southampton - subtotal    8936.3375
            First - subtotal                                   18017.4125
Second      Cherbourg         female                             176.8792
                              male                               254.2125
                              Second | Cherbourg - subtotal      431.0917
            Queenstown        female                              24.7000
                              male                                12.3500
                              Second | Queenstown - subtotal      37.0500
            Southampton       female                            1468.1500
                              male                              1865.5500
                              Second | Southampton - subtotal   3333.7000
            Second - subtotal                                   3801.8417
Third       Cherbourg         female                             337.9833
                              male                               402.1462
                              Third | Cherbourg - subtotal       740.1295
            Queenstown        female                             340.1585
                              male                               465.0458
                              Third | Queenstown - subtotal      805.2043
            Southampton       female                            1642.9668
                              male                              3526.3945
                              Third | Southampton - subtotal    5169.3613
            Third - subtotal                                    6714.6951
grand_total                                                    28533.9493

`sidetable` также позволяет настраивать уровни промежуточных итогов и итоговые метки.